# Digit Recognizer

In this kernel a Convolutional neural network is trained to recognized Handwritten digits and predict the same from the test set given

## Importing the necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

## Reading the training and test data

In [ ]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')
y_train = train_data.iloc[:,0].values
x_train = train_data.iloc[:,1:].values
x_test = test_data.iloc[:,:].values

In [ ]:
train_data.head(5)

## Function to display the image

In [ ]:
def row_calculator(number_of_images, number_of_columns):
    if number_of_images % number_of_columns != 0:
        return (number_of_images / number_of_columns)+1
    else:
        return (number_of_images / number_of_columns)

In [ ]:
def display_image(x, img_size, number_of_images):
    plt.figure(figsize = (8, 7))
    if x.shape[0] > 0:
        n_samples = x.shape[0]
        x = x.reshape(n_samples, img_size, img_size)
        number_of_rows = row_calculator(number_of_images, 4)
        for i in range(number_of_images):
            plt.subplot(number_of_rows, 4, i+1)
            plt.imshow(x[i])

In [ ]:
display_image(x_train, 28, 16)

In [ ]:
display_image(x_test, 28, 16)

In [ ]:
g = sns.countplot(y_train)

Since most of the labels have almost equal number of training examples, the entire dataset can be used

## Normalization and Encoding categorical variable

In [ ]:
x_train = x_train/255
x_test = x_test/255
y_train = to_categorical(y_train, num_classes = 10)

In [ ]:
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [ ]:
print(x_train.shape)
print(y_train.shape)

## Creating the Neural Network

In [ ]:
recognizer = Sequential()

recognizer.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
recognizer.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
recognizer.add(MaxPool2D(pool_size=(2,2)))
recognizer.add(Dropout(0.25))


recognizer.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
recognizer.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
recognizer.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
recognizer.add(Dropout(0.25))


recognizer.add(Flatten())
recognizer.add(Dense(units = 256, input_dim = 1024, activation = 'relu'))
recognizer.add(Dense(units = 256, activation = "relu"))
recognizer.add(Dense(units = 256, activation = 'relu'))
recognizer.add(Dropout(0.5))
recognizer.add(Dense(10, activation = "softmax"))

In [ ]:
recognizer.summary()

In [ ]:
optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
recognizer.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

In [ ]:
datagen.fit(x_train)

## Training the Neural Network

In [ ]:
recognizer.fit_generator(datagen.flow(x_train,y_train, batch_size=100), epochs = 30, steps_per_epoch=x_train.shape[0] // 100)

## Making predictions on the test data

In [ ]:
# predict results
results = recognizer.predict(x_test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_with_visualization.csv",index=False)